In [15]:
import numpy as np

from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, GRU
from keras.callbacks import ModelCheckpoint
from random import randint
from keras.models import model_from_json


In [16]:
with open("hazes.txt") as corpus_file:
    corpus = corpus_file.read()
    corpus = corpus.lower()
print("Loaded a corpus of {0} characters".format(len(corpus)))

Loaded a corpus of 41701 characters


In [17]:
from keras.models import model_from_json

# Build our network from loaded architecture and weights

from tensorflow import keras

with open('model.json', 'r') as file:
    data = file.read()

model = model_from_json(data)


model.load_weights('weights-20.hdf5')
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 20, 128)           86016     
                                                                 
 lstm_1 (LSTM)               (None, 20, 64)            49408     
                                                                 
 lstm_2 (LSTM)               (None, 20, 64)            33024     
                                                                 
 lstm_3 (LSTM)               (None, 20, 64)            33024     
                                                                 
 lstm_4 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 39)                2535      
                                                                 
 activation (Activation)     (None, 39)                0

In [18]:
# Get a unique identifier for each char in the corpus, then make some dicts to ease encoding and decoding
chars = sorted(list(set(corpus)))
num_chars = len(chars)
encoding = {c: i for i, c in enumerate(chars)}
decoding = {i: c for i, c in enumerate(chars)}

In [19]:
def sample(preds, temperature=1.0):
  if temperature <= 0:
    return np.argmax(preds)
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [20]:
def generate(seed_pattern):
        X = np.zeros((1, sentence_length, num_chars), dtype=bool)
        #print(X.shape)
        for i, character in enumerate(seed_pattern):
            X[0, i, encoding[character]] = 1
        
        generated_text = ""
        for i in range(500):
            # even de temperatuur toevoegen.
            prediction = sample(model.predict(X, verbose=0)[0],0.5)
            generated_text += decoding[prediction]

            activations = np.zeros((1, 1, num_chars), dtype=bool)
            activations[0, 0, prediction] = 1
            X = np.concatenate((X[:, 1:, :], activations), axis=1)

        return generated_text

def make_seed(seed_phrase=""):
        if seed_phrase:
            phrase_length = len(seed_phrase)
            pattern = ""
            for i in range (0, sentence_length):
                pattern += seed_phrase[i % phrase_length]
        else:
            seed = randint(0, corpus_length - sentence_length)
            pattern = abba_corpus[seed:seed + sentence_length]

        return pattern

In [21]:
sentence_length = 20
seed = make_seed('ik ben blij en ik je')
print(seed)
txt =  generate(seed)
print(txt)

ik ben blij en ik je
 nooit meer taop
tot mij tu liefen
de legt dat is een verdeen
dat jij ne snaal
toe ik jou niet meer anler van mij
het is wat jij niet onlen staat
met ik jou niet dat ik verdeen
want ik hou naoie ondiet
maar ik verdeen
noar os een je niet genoef
ik hieg met jij nu niet ze vrrrrtwe tij

met is verdt mee,
maar ik gaat me hier lonen mij verleen
maar de wat jij mag laar weer ondeen veiegen
ons jij nu niet garhen
maar nooit maar geven
die ik moet alleen boor jou staat
met ik vrar ne elleen

ik heb nie


## Sample song
In the bard and show you on your lovelight and i can't get the mowner i'm a marion an and every mind, there's a boot